<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
from src.nn import BACTERIA, Runner, ConfusionMatrixCallback, get_multimodel_components, metrics
logdir = './logs/segmentation'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
class MultiRunner(Runner):

    def _handle_batch(self, batch):
        global criterion

        x, y, z = batch
        y_pred, z_pred = self.model(x)
        is_train = self.state.stage_name == 'train'

        with torch.set_grad_enabled(is_train):
            loss_dice = criterion['dice'](y_pred, y)
            loss_crossentropy = criterion['crossentropy'](z_pred, z)
            loss = loss_dice + loss_crossentropy
            metric_iou = metrics.iou(y_pred, y)
            self.state.batch_metrics.update({
                'loss_dice': loss_dice,
                'loss_crossentropy': loss_crossentropy,
                'loss': loss,
                'metric_iou': metric_iou,
            })

            if is_train:
                loss.backward()
                self.state.optimizer.step()
                self.state.optimizer.zero_grad()

In [ ]:
dataset = BACTERIA(keys=['image', 'mask', 'label'])
experiments = list(dataset.crossval(kfold=4, batch_size=16))
num_experiment = 0
loaders = experiments[num_experiment]
model, optimizer, scheduler, criterion = get_multimodel_components('resnet34', 'adam', 'steplr')
model.train()
runner = MultiRunner()
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    callbacks=callbacks,
    loaders=loaders,
    logdir=f'{logdir}/{num_experiment}',
    num_epochs=100,
    verbose=True,
)

In [ ]:
! zip -r {logdir}.zip {logdir}
! git pull origin master
! git add logs
! git commit -m 'Changed from Colab'
! git push -u origin master